# Knowledge distillation implementation

Read about 5 more techniques for model compression here: [Machine Learning Model Compression: A Critical Step Towards Efficient Deep Learning](https://www.dailydoseofds.com/model-compression-a-critical-step-towards-efficient-machine-learning)


# Imports

In [2]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
import pandas as pd

from time import time
from tqdm import tqdm
from torch.utils.data import DataLoader

# Load the MNIST dataset

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100.0%
100.0%
100.0%
100.0%


# Knowledge Distillation

## Teacher Model

In [4]:
class TeacherNet(nn.Module):
    def __init__(self):
        super(TeacherNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(5, 5)
        self.fc1 = nn.Linear(32 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x        

## Evaluation function

In [5]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

## Initialize and train the teacher model

In [6]:
teacher_model = TeacherNet()
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
teacher_criterion = nn.CrossEntropyLoss()

In [7]:
for epoch in range(5):
    teacher_model.train()
    running_loss = 0.0
    
    for data in trainloader:
        inputs, labels = data
        teacher_optimizer.zero_grad()
        outputs = teacher_model(inputs)
        loss = teacher_criterion(outputs, labels)
        loss.backward()
        teacher_optimizer.step()
        
        running_loss += loss.item()
        
    teacher_accuracy = evaluate(teacher_model)
        
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, Accuracy: {teacher_accuracy * 100:.2f}%")

Epoch 1, Loss: 0.23124422427656047, Accuracy: 97.58%
Epoch 2, Loss: 0.07545049265890853, Accuracy: 98.38%
Epoch 3, Loss: 0.057798394025849506, Accuracy: 97.82%
Epoch 4, Loss: 0.048843113230049875, Accuracy: 98.43%
Epoch 5, Loss: 0.04173276223664754, Accuracy: 98.81%


## Student Model

In [9]:
class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Initialize and train the student model

In [10]:
student_model = StudentNet()
student_optimizer = optim.Adam(student_model.parameters(), lr=0.001)

## Loss function (KL Divergence)

In [11]:
def knowledge_distillation_loss(student_logits, teacher_logits):
    p_teacher = F.softmax(teacher_logits , dim=1)
    p_student = F.log_softmax(student_logits, dim=1)
    loss = F.kl_div(p_student, p_teacher, reduction='batchmean')
    return loss

In [14]:
# Train the student model with knowledge distillation
for epoch in range(5):  # You can adjust the number of epochs
    student_model.train()
    running_loss = 0.0
    
    for data in trainloader:
        inputs, labels = data
        student_optimizer.zero_grad()
        student_logits = student_model(inputs)
        teacher_logits = teacher_model(inputs).detach()  # Detach the teacher's output to avoid backpropagation
        loss = knowledge_distillation_loss(student_logits, teacher_logits)
        loss.backward()
        student_optimizer.step()
        
        running_loss += loss.item()
    
    student_accuracy = evaluate(student_model)
        
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(testloader)}, Accuracy: {student_accuracy * 100:.2f}%")


KeyboardInterrupt: 

In [13]:
%timeit evaluate(teacher_model)

2.32 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit evaluate(student_model) # student model runs faster

1.88 s ± 82.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Read about 5 more techniques for model compression here: [Machine Learning Model Compression: A Critical Step Towards Efficient Deep Learning](https://www.dailydoseofds.com/model-compression-a-critical-step-towards-efficient-machine-learning)